In [1]:
from paddlenlp.dataaug import WordSubstitute

c:\Users\zjt78\miniconda3\envs\faq\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
s1 = "工地买了建工险，出了事故多长时间上报保险公司有效。"
aug = WordSubstitute('synonym', create_n=3, aug_percent=0.3)
augmented = aug.augment(s1)
print("origin:", s1)
print("augmented:", augmented)

100%|██████████| 46328/46328 [00:05<00:00, 7956.66it/s] 


origin: 工地买了建工险，出了事故多长时间上报保险公司有效。
augmented: ['工地买了基建工险，出收故多长时间上报保险公司有效。', '工地买了建工险，出完结故多长时间上报保险商行有效。', '工地买了建工险，出收摊儿故多长光阴上报保险公司有效。']


In [3]:
import paddle

In [8]:
paddle.set_device("gpu")

Place(gpu:0)

In [9]:
aug = WordSubstitute('mlm', create_n=2)
augmented = aug.augment(s1)
print("origin:", s1)
print("augmented:", augmented[0])

[2023-02-22 18:08:38,955] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForMaskedLM'> to load 'ernie-1.0'.
[2023-02-22 18:08:38,957] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 513,
  "model_type": "ernie",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "paddlenlp_version": null,
  "pool_act": "tanh",
  "task_id": 0,
  "task_type_vocab_size": 3,
  "type_vocab_size": 2,
  "use_task_id": true,
  "vocab_size": 18000
}

[2023-02-22 18:08:38,960] [    INFO] - Configuration saved in C:\Users\zjt78\.paddlenlp\models\ernie-1.0\config.json


RuntimeError: (PreconditionNotMet) The third-party dynamic library (cudnn64_8.dll) that Paddle depends on is not configured correctly. (error code is 126)
  Suggestions:
  1. Check if the third-party dynamic library (e.g. CUDA, CUDNN) is installed correctly and its version is matched with paddlepaddle you installed.
  2. Configure third-party dynamic library environment variables as follows:
  - Linux: set LD_LIBRARY_PATH by `export LD_LIBRARY_PATH=...`
  - Windows: set PATH by `set PATH=XXX; (at ..\paddle\phi\backends\dynload\dynamic_loader.cc:305)
